# 5. Timing analysis

We will produce a light curve of Mrk 421 in two energy bands, compute the ratio of the fluxes and see if there is any hint of spectral variability.

For this tutorial, we will need a few extra python packages (such as `astroquery` to query Virtual Observatory services).

In [ ]:
# !mamba install -c conda-forge astroquery
#
# or:
#
# !mamba create -n cads-2024 -c conda-forge gammapy=1.2 ipykernel astroquery tqdm

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style
style.use('tableau-colorblind10')

import numpy as np

from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.time import Time
from astroquery.simbad import Simbad

import scipy


from gammapy.data import DataStore
from gammapy.datasets import Datasets, SpectrumDataset, SpectrumDatasetOnOff
from gammapy.estimators import FluxPointsEstimator, LightCurveEstimator
from gammapy.estimators.utils import (
    compute_lightcurve_fvar,
    compute_lightcurve_fpp,
    compute_lightcurve_doublingtime,
    get_rebinned_axis,
    resample_energy_edges,
)
from gammapy.makers import (
    DatasetsMaker,
    SafeMaskMaker,
    SpectrumDatasetMaker,
    ReflectedRegionsBackgroundMaker,
)
from gammapy.makers.utils import make_theta_squared_table
from gammapy.maps import MapAxis, RegionGeom, WcsGeom
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    PointSpatialModel,
    SkyModel,
)
from gammapy.utils import pbar
pbar.SHOW_PROGRESS_BAR = True
from gammapy.visualization import plot_spectrum_datasets_off_regions, plot_theta_squared_table

from regions import CircleSkyRegion

from IPython.display import display

We first load the relevant data set:

In [ ]:
data_store = DataStore.from_dir(
    f"../../../CTA-SDC-school-20241010T210126Z-001/CTA-SDC-school"
)

We set the properties of the source of interest. *Bonus*: we can use Virtual Observatory services to query some other source parameters, like its redshift.

In [ ]:
src = dict()
src['Name'] = 'Mrk 421'
src['Position'] = SkyCoord.from_name(src['Name'])

try:
    simbad = Simbad()
    simbad.add_votable_fields("z_value")
    query = simbad.query_object(src['Name'])
    src['Redshift'] = query["Z_VALUE"].data[0]
except NameError:
    print("Cannot use Simbad, will set the source redshift manually")
    src['Redshift'] = 0.030

We select a sub-sample of data acquired on our source:

In [ ]:
selection = dict(
    type="sky_circle",
    frame="icrs",
    lon=src['Position'].ra,
    lat=src['Position'].dec,
    radius="3 deg",
)
selected_obs_table = data_store.obs_table.select_observations(selection)

obs_ids = selected_obs_table["OBS_ID"]
observations = data_store.get_observations(obs_ids)

Let's create a time intervals, for later use, and filter the observations on it:

In [ ]:
t0 = XXX
duration = 20 * u.min
n_time_bins = XXX
times = t0 + np.arange(n_time_bins) * duration
time_intervals = XXX

short_observations = observations.select_time(time_intervals)

# Data reduction

Let's perform a 1D analysis of the data.

In [ ]:
on_region_radius = Angle("0.1 deg")

on_region = CircleSkyRegion(center=src['Position'], radius=on_region_radius)
exclusion_region = CircleSkyRegion(center=src['Position'],radius=0.5 * u.deg)
geom = WcsGeom.create(
    npix=(120, 120), binsz=0.05, skydir=src['Position'], proj="TAN", frame="icrs"
)
exclusion_mask = ~geom.region_mask([exclusion_region])

In [ ]:
energy_axis = XXX
energy_axis_true = XXX

geom = RegionGeom.create(region=on_region, axes=[energy_axis])

dataset_empty = XXX
dataset_maker = XXX

bkg_maker = XXX
safe_mask_maker = XXX

In [ ]:
%%time

# Parallel version
makers = [dataset_maker, bkg_maker, safe_mask_maker]  # the order matters
datasets_maker = DatasetsMaker(makers, stack_datasets=False, n_jobs=6)
datasets = datasets_maker.run(dataset_empty, observations)

In [ ]:
dataset_stack = datasets.stack_reduce()

Let's find some energy at which to slice our data, to build two well-balanced sets into two energy bands:

In [ ]:
excess = dataset_stack.excess.data.T[0][0]
mask = excess > 0
excess = excess[mask]
energy_bins = dataset_stack.counts.geom.axes['energy'].center[mask]
split_value = 2./3. * np.sum(excess)
split_mask = np.cumsum(excess) > split_value
e_split = energy_bins[split_mask][0]

print(f'Split energy: {e_split:.3f}')

In [ ]:
e_min = dataset_stack.energy_range_safe[0].data[0][0] * dataset_stack.energy_range_safe[0].unit
e_max = dataset_stack.energy_range_safe[-1].data[0][0] * dataset_stack.energy_range_safe[0].unit
print(f'Energy threshold: {e_min:.3f}')
print(f'Maximal energy: {e_max:.3f}')

# Fit overall spectrum

In [ ]:
spectral_model = XXX

In [ ]:
source = SkyModel(spectral_model=spectral_model,
                 name=src["Name"])

for ds in datasets:
    ds.models = source

In [ ]:
%%time

fit_joint = Fit()
result_joint = fit_joint.run(datasets=datasets)

# we make a copy here of the optimised model for later use
model_best_joint = source.copy(name=src["Name"])

In [ ]:
print(result_joint)
display(result_joint.models.to_parameters_table())

In [ ]:
%%time

# Compute flux points

energy_edges = XXX

fpe = FluxPointsEstimator(XXX)
flux_points = fpe.run(datasets)

Let's plot the SED so far:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

plot_kwargs = {
    "energy_bounds": [e_min, e_max],
    "sed_type": "e2dnde",
    "yunits": u.Unit("erg cm-2 s-1"),
    "ax": ax,
}

XXXX

# Light curve

Let's compute the light curve in two energy ranges

In [ ]:
energy_lo = [e_min.value, e_split.value] * e_min.unit
energy_hi = [e_split.value, e_max.value] * e_max.unit
energy_all = [e_min.value, e_max.value] * e_min.unit

e_ranges = [energy_lo, energy_hi, energy_all]

In [ ]:
lc_maker = LightCurveEstimator(XXX)
lc = lc_maker.run(datasets)

Plot the light curves:

In [ ]:
XXX

We can also rebin the light curve to larger, fixed time bins, or requesting a minimum TS.
As an example, let's create another light curve for the whole energy range, and play with it.

In [ ]:
axis_new = get_rebinned_axis(XXX)
print(axis_new)

# Resample the original light curve
XXX

# Plot the original and resampled light curves
plt.figure(figsize=(8, 6))
XXX

# Fractional and point-to-point variability

In [ ]:
# Compute the global fractional variability, for each energy intervals
XXX

In [ ]:
# Compute the point-to-point fractional variability, for each energy intervals
XXX

In [ ]:
# Compute the characteristic doubling time of the light curves, for each energy intervals
XXX

# Hardness ratio diagrams

Let's compute the flux ratio of our two light curves, plot them against time, and against the overall flux (i.e. hardness ratio diagram).

Access the low-energy and high-energy light curves.

**Tip**: Remember that `RegionNDMap` holds quantities of `numpy.ndarray`s.

In [ ]:
lc_lo = XXX
lc_hi = XXX

lc_tot = XXX

In [ ]:
flux_ratio = XXX

In [ ]:
# Plot the flux ratio versus time

XXX

Now, let's plot the hardness ratio diagram (integral flux in the whole energy range, versus the flux ratio)

In [ ]:
XXX